# 1. Spark运行构架
1. 符合主从结构 （**master-slave**）
2. 驱动器节点（**Driver**）:负责中央协调，调度各个分布式工作节点
3. 执行器节点（**Executor**）: 工作节点
4. 通过一个集群管理器（**Cluster Manager**）的外部服务在集群中机器上启动

## 1.1 驱动器节点
1. 是执行你程序中main()方法的进程，它执行用户编写的用来创建SparkContext，创建RDD,以及进行RDD转化操作和行动操作的代码
2. 两个职责：
   - 把用户的程序转为任务
     - Diver负责把用户程序转化为多个物理执行的单元，这些单位被称为**任务(task)**
     - Spark程序隐式的创建出一个由操作组成的逻辑上的有向无环图（**Directed Acyclic Graph, DAG**）,当Driver启动时，会把DAG转化为物理执行几乎
     - Spark把逻辑执行计划转化成一系列的步骤（**stage**），每个stage包含很多的task，这些task被打包送入cluster执行
     - task是Spark中最小的工作单位
   - 为Executor调度任务
     - Driver必须在各Executor进程间协调任务的调度
     - Executor启动之后，会向Driver注册自己，所以Driver始终有该应用中所有的Executor的完整记录（**每个Executor代表一个能够处理task和存储RDD的进程**）
     - Driver会根据当前的Executor集合，尝试吧所有的task基于数据的存储位置分配给合适的Executor，在task执行时，Executor会把缓存的数据存储起来，Driver同样会跟踪这些缓存的数据的位置，并根据此来调度之后的task，**尽量来减少数据的网络传输**
     
## 1.2 执行器节点
1. **Spark中Excutor其实是一种工作进程**,负责在Spark作业中执行task，**各task相互独立**
2. Spark应用启动时，Executor就同时被启用了，并且始终伴随着整个Spark应用的生命周期而存在。
3. 如果有Executor发生了异常和崩溃，Spark应用也可以继续执行。
4. Executor的两个作用：
   - 负责组成Spark应用的任务，并将结果返回给Driver进程
   - 通过自身的块管理器(**Block Manager**)为用户程序中要求缓存的RDD提供内存是存储
5. **RDD是直接缓存在Executor进程内的，因此可以充分利用缓存数据加速运算**


# 2. 集群管理器 Cluster Manager
1. Spark依赖于集群管理器来启动Executor，在某些特殊情况下，也依赖集群管理器来启动Driver
2. 集群管理器是Spark中的可插拔式组件。

## 2.1 spark-submit
- **不论使用哪一种集群管理器，都可以使用spark-submit将你的应用提交到那种集群管理器上。通过不同的配置选项，spark-submit可以连接到相应的集群管理器上，并控制应用所使用的资源数量**
- spark-submit脚本启动Driver，调用用户定义的main()方法。
- 如果Driver的main()方法退出或者调用了SparkContext.stop()，Driver会终止Executor，并通过集群管理器释放资源

# 3. spark-submit部署应用
1. **Spark为各种集群管理器提供了统一的工具来提交作业，这个工具就是spark-submit脚本**
    - EX: 提交Python应用
    ```
    bin/spark-submit my_script.py
    ```
    - 如果调用spark-submit除了脚本名或者JAR包外没有别的参数，则这个程序只会在本地执行
    - EX: 含有参数的提交
    ```
    bin/spark-submit --master spark://host:7077 --executor -memory 10g my_script.py
    ```
    - <img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/9c329cc1fd1d128bb6b35e948e884acd4363d441/ipynbFiles/Materials/spark_submit_master%E5%8F%AF%E4%BB%A5%E6%8E%A5%E5%8F%97%E7%9A%84%E5%80%BC.jpg" />
    - spark-submit提供两大类选项：
        - 第一类：调度信息，比如上例中的为作业申请的资源量（10G内存）
        - 第二类：应用运行时候的依赖，比如需要部署到所有工作节点上的库和文件
    - spark-submit的一般格式
    ```
    bin/spark-submit [option] <app jar | python file> [app options]
    ```
    - [options]是要传给spark-summit的标记列表
        - 可以运行spark-submit --help列出所有的可接受标记
        - <img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/7b39b328eeb559a07a96f0c7c6bf0ea421581834/ipynbFiles/Materials/spark_submit%E7%9A%84%E4%B8%80%E4%BA%9B%E5%B8%B8%E8%A7%81%E6%A0%87%E8%AE%B0.jpg" />
    - 《app jar | python file》 表示包含的应用入口的JAR包或Python脚本
    - [app options]是传给应用的选项
    - spark-submit还允许通过 --conf prop=value来设置任意的SparkConf配置选项
    - 也可以使用 --properties-File指定一个包含键值对的属性文件

# 4, 打包代码与依赖
## 4.1 Python 用户
- **由于PySpark使用工作节点机器上已有的Python环境**
- 可以使用pip,easy_install等包管理器在集群所有的机器中安装的所需要的库
- 也可以把依赖手动安装到Python安装目录下的site-packages目录中
- 也可以使用spark-submit的 --py-Files参数提交独立的库，这样它们也会被添加到Python解释器的路径中。

## 4.2 Java和Scala用户
- Java工程的包构建工具 -- Maven
- Scala工程的包构建工具 -- sbt

## 4.3 依赖冲突
- 当用户应用与Spark本身依赖同一个库时可能会发生依赖冲突，导致程序崩溃
- 一般此时会抛出*NoSuchMethodError,ClassNotFoundException*或者其他类加载相关的JVM异常。
- 解决方法有两种：
   - 修改自己的应用，使其使用的依赖库的版本与Spark使用的相同
   - "shading"方式打包你的应用，“shading”可以让你以另一个命名空间保留冲突的包，并自动重写应用的代码使得它们使用重命名后版本

# 5, Spark应用内与应用间的调度
1. 多用户的集群，一般执行的优先级有相应的集群管理器决定
2. 长期运行(long lived)应用的任务调度一般由公平调度器(Fair Scheduler)完成

# 6, 集群管理器
## 6.1 独立集群管理器 （Standalone Cluster Manager）
- 由一个主节点和几个工作节点组成，各自都分配一定量的内存和cpu核心，提交应用时，配置好Executor进程使用的内存量，和所有Executor进程使用的CPU核心综述
- 启动：
   - 编译好的Spark文件复制到所有机器的相同目录之下
   - 设置好每个机器之前的SSH无密码登录
   - 编辑主节点上conf/slaves文件并填上所有节点的主机名
   - 在主节点上运行sbin/start-all.sh,可以再特定网页上查看用户界面，一般网址为：http://masternode:8080
   - 要停止集群，在主节点上运行bin/stop-all.sh
- 提交应用
   - 使用spark-submit命令
   - EX: 集群模式下启动PySpark Shell,在网页上查看其详细信息
       ```
       pyspark --master spark://zeus:7077
       ```
   - 常见错误：为Executor申请的内存超过了集群所能提供的内存的总量，此时，独立集群管理始终无法分配Executor
   - 支持两种部署模式：
      - client模式（默认），Driver运行在提交spark-submit命令的机器上，需要很高的带宽和机器性能
      - cluster模式，Driver会作为某工作节点上一个独立的进程运行在独立集群管理器的内部，主节点也会被申请执行器节点
         - 可以在执行spark-submit时用 --deploy-mode cluster参数切换到集群模式
- 配置资源用量
   - 应对多应用共享spark集群的情况，决定如何在Executor之间分配资源。
   - 独立集群管理器使用最基础的调度策略，这种策略允许限制各个应用的用来来让多个应用并发执行。Apache Mesos支持运行时动态的资源共享，而YARN则有分级队列的概念，可以让你限制不同类别的应用用量
   - 资源分配靠下面两个设置来控制：
       - Executor内存，spark-submit的 --executor-memoey参数来配置
         - 一个工作节点只能有一个Executor(但是一台机器可以有很多工作节点），默认为1GB
       - 占用核心总数的最大值
         - 一个应用中所有Executor所占用的核心综述，默认无限，即应用可以在集群所有的可用的节点上启动Executor.
         - 对于多用户来说，需要设置限制，可以通过spark-submit的 --total-executorcores参数来设置，或者是Spark配置文件中设置的spark.cores.max值
         - 可以通过网页查看
- 独立集群管理器默认会为每个应用使用尽可能分散的Executor进程
   - EX: 一个集群20台物理节点，每台4核，当执行 --executor-memory 1G 和 -- total-executor-cores 8 时，Spark会在每个机器上启动8个Executor,每个内存1GB
   - 可以通过设置，是独立集群管理器默认选择最少的节点（即只选两台2机器）
         

   

## 6.2 Hadoop YARN
- 详细查阅SparK官方文档：(http://spark.apache.org/docs/latest/submitting-applications.html)
## 6.3 Apache Mesos
## 6.4 Amazon EC2

# 7 选择合适的集群管理器
- 从零开始，先选择独立集群管理器，安装简单，如果只是使用spark的话，独立集群管理器可以提供其他集群管理器完全一样的功能
- 如果要使用Spark其他的应用一起使用，则可以考虑YARN和MESOS
- **在任何时候，最好宝Spark运行在HDFS的节点上，这样能快速访问存储。**
